In [1]:
import numpy as np
import pandas as pd
import glob
import geopy
import geopy.distance
import datetime
import sys
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = r"/content/drive/My Drive/ship_data_files /train_ship_data_final_classifier.csv"
df=pd.read_csv(path,sep=',',low_memory=False)
df

Mounted at /content/drive


,Length,Breadth,Draught,Longitude,Latitude,SOG,bool,ETA
0,152,24,6.8,5.838777,53.63944,14.3,0,12.685507
1,152,24,6.8,5.845800,53.64076,14.3,0,12.667177
2,152,24,6.8,5.852149,53.64198,14.3,0,12.650496
3,152,24,6.8,5.858515,53.64320,14.3,0,12.633836
4,152,24,6.8,5.864876,53.64441,14.3,0,12.617179
...,...,...,...,...,...,...,...,...
186520,229,38,13.3,9.806900,53.55355,7.4,1,0.083519
186521,229,38,13.3,9.810224,53.55293,7.3,1,0.066774
186522,229,38,13.3,9.813442,53.55230,7.2,1,0.049974
186523,229,38,13.3,9.816650,53.55172,7.1,1,0.033443


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
df

,Length,Breadth,Draught,Longitude,Latitude,SOG,bool,ETA
0,152,24,6.8,5.838777,53.63944,14.3,0,12.685507
1,152,24,6.8,5.845800,53.64076,14.3,0,12.667177
2,152,24,6.8,5.852149,53.64198,14.3,0,12.650496
3,152,24,6.8,5.858515,53.64320,14.3,0,12.633836
4,152,24,6.8,5.864876,53.64441,14.3,0,12.617179
...,...,...,...,...,...,...,...,...
186520,229,38,13.3,9.806900,53.55355,7.4,1,0.083519
186521,229,38,13.3,9.810224,53.55293,7.3,1,0.066774
186522,229,38,13.3,9.813442,53.55230,7.2,1,0.049974
186523,229,38,13.3,9.816650,53.55172,7.1,1,0.033443


In [4]:
#Training part of the data
from sklearn import metrics

from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from hyperopt import fmin, tpe, hp,STATUS_OK, Trials
import matplotlib.pyplot as plt
!pip install tpot
from tpot import TPOTClassifier

     |████████████████████████████████| 92kB 7.0MB/s 
     |████████████████████████████████| 163kB 20.4MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=4670b2a056b483f13954ff1e5f376f3617ebe6aeae9a63925266a68d5a488ae3
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [5]:
#Features
features = df.drop('bool', axis=1)
features= np.array(features)

# Labels
labels= np.array(df['bool'])

feature_list = list(df.columns)

In [6]:
#Splitting the data to train and test

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [7]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:',  y_test.shape)



Training Features Shape: (139893, 7)
Training Labels Shape: (139893,)
Testing Features Shape: (46632, 7)
Testing Labels Shape: (46632,)


In [8]:
# Training part with logistic regression without tuning(Default values)

lr = LogisticRegression()

start_time = time.time()

lr.fit(X_train, y_train)

sv_notune= time.time() - start_time

y_pred= lr.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7864556527706296


In [9]:
# SVM object to be used to tune the default kernel will be 'rbf' 

lrc = LogisticRegression()



In [10]:
lrc.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [11]:
# Tuning LRC with the following algorithims
        # 1) Random
        # 2) Bayesian
        #  3) Genetic algorithims
        
# Random searcch set parameters
C=np.random.uniform(0.01,100,20)
max_iter = np.random.randint(10,100,20)
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty= ['none', 'l1', 'l2', 'elasticnet']
random_grid = {'C': C, 'solver':solver,
               'penalty': penalty,'max_iter':max_iter
               }
random_grid 

{'C': array([53.84288914, 62.4440362 , 10.25109884, 85.88505026, 65.60444484,
         2.49029933, 53.59361721, 55.73900889, 20.33127667, 62.67267622,
        52.57734407,  6.03542445, 74.56999338, 91.75277805, 55.03006272,
         3.61922943, 13.11952969, 44.34376643, 94.54764051, 60.48132702]),
 'max_iter': array([43, 48, 20, 41, 88, 15, 78, 82, 39, 87, 57, 39, 90, 15, 37, 83, 75,
        29, 50, 96]),
 'penalty': ['none', 'l1', 'l2', 'elasticnet'],
 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

In [12]:
#Random search tuning
lr_random = RandomizedSearchCV(estimator = lrc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=10, random_state=42, n_jobs =-1)
start_time = time.time()
lr_random.fit(X_train, y_train)

random_time =time.time() - start_time

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  3

In [13]:
#Parameters selected for random search
lr_random.get_params()


{'cv': 3,
 'error_score': nan,
 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': False,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__l1_ratio': None,
 'estimator__max_iter': 100,
 'estimator__multi_class': 'auto',
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__solver': 'lbfgs',
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'iid': 'deprecated',
 'n_iter': 100,
 'n_jobs': -1,
 'param_distributions': {'C': array([53.84288914, 62.4440362 , 10.25109884, 85.88505026, 65.60444484,
          2.4

In [14]:
#Testing on the test features, predictions for random search
y_pred1=lr_random.predict(X_test)

print("%s seconds it took to train with Random search tuning" % (random_time))
print(classification_report(y_test,y_pred1))

449.4774467945099 seconds it took to train with Random search tuning
              precision    recall  f1-score   support

           0       0.83      0.96      0.89     35954
           1       0.73      0.36      0.48     10678

    accuracy                           0.82     46632
   macro avg       0.78      0.66      0.69     46632
weighted avg       0.81      0.82      0.80     46632



In [15]:
#Tuning with Bayesian optimization

search_space= {'C': hp.uniform ('C', 0.01, 100),
        'max_iter': hp.uniform ('max_iter', 10, 100)
    }
def objective(search_space):
    bayes = LogisticRegression(C = search_space['C'],max_iter = search_space['max_iter'])
    accuracy = cross_val_score(bayes, X_train, y_train, cv = 4).mean()
    return {'loss': accuracy, 'status': STATUS_OK }

trials = Trials()


# Training from bayes tuning


start_time = time.time()


best = fmin(fn= objective,
            space= search_space,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)
best



trainedlrc  = LogisticRegression(C = best['C'],max_iter = best['max_iter']).fit(X_train, y_train)

bayesian_time =time.time() - start_time

y_pred2 = trainedlrc.predict(X_test)

  2%|▏         | 1/50 [00:03<03:14,  3.96s/it, best loss: 0.7902539825545656]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

  4%|▍         | 2/50 [00:07<03:02,  3.81s/it, best loss: 0.7902539825545656]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

  6%|▌         | 3/50 [00:11<03:04,  3.93s/it, best loss: 0.7902539825545656]

...

 98%|█████████▊| 49/50 [01:55<00:02,  2.46s/it, best loss: 0.7739558091502005]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

100%|██████████| 50/50 [01:58<00:00,  2.37s/it, best loss: 0.7739558091502005]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [19]:
# The genetic algorithim part

lrc_tpot = TPOTClassifier(verbosity=10, 
 scoring='accuracy', 
 random_state=20, 
 n_jobs=-1, 
 generations=5, 
 population_size=10,
 early_stop=5)

start_time = time.time()

lrc_tpot.fit(X_train, y_train) 

genetic_time = time.time() - start_time

32 operators have been imported by TPOT.


_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 89.
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
Skipped pipeline #39 due to time out. Continuing to the next pipeline.
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only dual=False, got dual=True.
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Skipped pipeline #50 due to time out. Continuing to the next pipeline.
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of 

In [20]:
predictions3 = lrc_tpot.predict(X_test)

In [21]:
# Results of training (time and classification report)

#1 Result of SVC without tuning
print("%s seconds it took to train without tuning" % (sv_notune))    
print(classification_report(y_test,y_pred))

#2 Result of SVC with Random search tuning
print("%s seconds it took to train with Random search tuning" % (random_time))
print(classification_report(y_test,y_pred1))

#3 Result of SVC with Bayesian search tuning 
print("%s seconds it took to train with Bayesian search tuning " % (bayesian_time))
print(classification_report(y_test,y_pred2))

#4 Result of Classification with Genetic Algorithim
print("%s seconds it took to train genetic algorithim" % (genetic_time))
print(classification_report(y_test,predictions3))


1.1356086730957031 seconds it took to train without tuning
              precision    recall  f1-score   support

           0       0.79      0.98      0.88     35954
           1       0.66      0.14      0.23     10678

    accuracy                           0.79     46632
   macro avg       0.73      0.56      0.55     46632
weighted avg       0.76      0.79      0.73     46632

449.4774467945099 seconds it took to train with Random search tuning
              precision    recall  f1-score   support

           0       0.83      0.96      0.89     35954
           1       0.73      0.36      0.48     10678

    accuracy                           0.82     46632
   macro avg       0.78      0.66      0.69     46632
weighted avg       0.81      0.82      0.80     46632

118.51793050765991 seconds it took to train with Bayesian search tuning 
              precision    recall  f1-score   support

           0       0.77      1.00      0.87     35954
           1       0.00      0.00   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
